![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)
# Vector Search with RedisVL

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/vector-search/01_redisvl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Prepare data

In this examples we will load a list of movies with the following attributes: `title`, `rating`, `description`, and `genre`.

We will embed the movie description so that user's can search for movies that best match the kind of movie that they're looking for.

**If you are running this notebook locally**, FYI you may not need to perform this step at all.

In [4]:
# NBVAL_SKIP
!git clone https://github.com/redis-developer/redis-ai-resources.git temp_repo
!mv temp_repo/python-recipes/vector-search/resources .
!rm -rf temp_repo

Cloning into 'temp_repo'...
remote: Enumerating objects: 797, done.
remote: Counting objects: 100% (337/337), done.
remote: Compressing objects: 100% (206/206), done.
remote: Total 797 (delta 248), reused 147 (delta 131), pack-reused 460 (from 3)
Receiving objects: 100% (797/797), 58.59 MiB | 23.53 MiB/s, done.
Resolving deltas: 100% (357/357), done.
mv: cannot move 'temp_repo/python-recipes/vector-search/resources' to './resources': Directory not empty


## Packages

In [2]:
%pip install -q "redisvl>=0.6.0" sentence-transformers pandas nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.3/152.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.7/278.7 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.8 MB/s eta 0:00:00


## Install Redis Stack

Later in this tutorial, Redis will be used to store, index, and query vector
embeddings created from PDF document chunks. **We need to make sure we have a Redis
instance available.

#### For Colab
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [3]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [5]:
import os
import warnings

warnings.filterwarnings('ignore')

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

### Create redis client

In [6]:
from redis import Redis

client = Redis.from_url(REDIS_URL)
client.ping()

True

In [7]:
#client.flushall()

### Load Movies Dataset

In [ ]:
data = [
    {
        "id": 1,
        "title": "סושי במסעדת יאמאטה",
        "description": "הצצנו מאחורי הקלעים של יאמאטה בתל אביב – מנת סשימי חדשה, אווירה אותנטית, ותחושה יפנית אמיתית.",
        "tags": "יאמאטה,סושי,יפני,תל אביב,מסעדה",
        "rating": 9
    },
    {
        "id": 2,
        "title": "המבורגר אמריקאי בשינקין",
        "description": "המבורגר עסיסי בסגנון דיינר עם גבינה נמסה וצ'יפס קראנצ'י. המקום הכי חם בשינקין!",
        "tags": "המבורגר,אמריקאי,fast food,שינקין,תל אביב",
        "rating": 8
    },
    {
        "id": 3,
        "title": "ראמן בדאון טאון חיפה",
        "description": "מסעדת רחוב שמגישה ראמן חם עם טעמים עשירים. חוויה אסייתית בלב חיפה.",
        "tags": "ראמן,יפני,חיפה,אסיאתי,street food",
        "rating": 7
    },
    {
        "id": 4,
        "title": "סושי ביפו העתיקה",
        "description": "סיבוב קולינרי בשוק הפשפשים – טעמנו רולים פריכים עם טונה חריפה ואבוקדו.",
        "tags": "סושי,יפו,תל אביב,מסעדה,רול",
        "rating": 8
    },
    {
        "id": 5,
        "title": "Tuna Bowl at Toki",
        "description": "Healthy tuna poke bowl with avocado and mango salsa at Toki, a stylish spot in Sarona.",
        "tags": "poke,tuna,sarona,tel aviv,healthy",
        "rating": 9
    }
]

df = pd.DataFrame(data)
df


In [8]:
import pandas as pd
import numpy as np
import json

# df = pd.read_json("resources/movies.json")
print("Loaded", len(df), "movie entries")

df.head()

Loaded 20 movie entries


,id,title,genre,rating,description
0,1,Explosive Pursuit,action,7,A daring cop chases a notorious criminal acros...
1,2,Skyfall,action,8,James Bond returns to track down a dangerous n...
2,3,Fast & Furious 9,action,6,Dom and his crew face off against a high-tech ...
3,4,Black Widow,action,7,Natasha Romanoff confronts her dark past and f...
4,5,John Wick,action,8,A retired hitman seeks vengeance against those...


In [9]:
from redisvl.utils.vectorize import HFTextVectorizer
from redisvl.extensions.cache.embeddings import EmbeddingsCache

os.environ["TOKENIZERS_PARALLELISM"] = "false"


hf = HFTextVectorizer(
    model="sentence-transformers/all-MiniLM-L6-v2",
    cache=EmbeddingsCache(
        name="embedcache",
        ttl=600,
        redis_client=client,
    )
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# Generate vectors
df["vector"] = hf.embed_many(df["description"].tolist(), as_buffer=True)

df.head()

,id,title,genre,rating,description,vector
0,1,Explosive Pursuit,action,7,A daring cop chases a notorious criminal acros...,b'\x9bf|=\na\n;\xbf\x91\xb7;\x19\xcb~\xbd\xd9d...
1,2,Skyfall,action,8,James Bond returns to track down a dangerous n...,b'\x9aD\x9e\xbd0\x9b\x89\xbc\xc3\x16\x95\xbc\x...
2,3,Fast & Furious 9,action,6,Dom and his crew face off against a high-tech ...,"b'*\xa5\xc7\xbc\xf6,\xa2=?\x19H\xbcK\xc6t\xbd\..."
3,4,Black Widow,action,7,Natasha Romanoff confronts her dark past and f...,b'u\xeb\x85\xbd\x0e\xcdo\xbd&\xe8\xc2\xbb6\xcf...
4,5,John Wick,action,8,A retired hitman seeks vengeance against those...,b'\xaf<x\xbb\xfb.\xc5=B\x86:;\xce\xd0\x94<\xf9...


## Define Redis index schema

In [11]:
from redisvl.schema import IndexSchema
from redisvl.index import SearchIndex


index_name = "movies"

schema = IndexSchema.from_dict({
  "index": {
    "name": index_name,
    "prefix": index_name,
    "storage_type": "hash"
  },
  "fields": [
    {
        "name": "title",
        "type": "text",
    },
    {
        "name": "description",
        "type": "text",
    },
    {
        "name": "genre",
        "type": "tag",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "rating",
        "type": "numeric",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "vector",
        "type": "vector",
        "attrs": {
            "dims": 384,
            "distance_metric": "cosine",
            "algorithm": "flat",
            "datatype": "float32"
        }
    }
  ]
})


index = SearchIndex(schema, client)
index.create(overwrite=True, drop=True)

In [12]:
!rvl index info -i movies -u {REDIS_URL}



Index Information:
╭───────────────┬───────────────┬───────────────┬───────────────┬───────────────┬╮
│ Index Name    │ Storage Type  │ Prefixes      │ Index Options │ Indexing      │
├───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼┤
| movies        | HASH          | ['movies']    | []            | 0             |
╰───────────────┴───────────────┴───────────────┴───────────────┴───────────────┴╯
Index Fields:
╭─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬╮
│ Name            │ Attribute       │ Type            │ Field Option    │ Option Value    │ Field Option    │ Option Value    │ Field Option    │ Option Value    │ Field Option    │ Option Value    │
├─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼──

## Populate index

In [13]:
index.load(df.to_dict(orient="records"))

['movies:01K17WBQ9GS6XF9GDPFPSS342V',
 'movies:01K17WBQ9G1AAFZ0CYK44CEMMZ',
 'movies:01K17WBQ9GNQXSNZ4Y152XMS2R',
 'movies:01K17WBQ9GD4RSBXDPFGJ5YMP6',
 'movies:01K17WBQ9G36TWXMK5FWTYFJ4D',
 'movies:01K17WBQ9GFVP42C4RVN880ZTR',
 'movies:01K17WBQ9GJZCTSJ63Z4D75VX0',
 'movies:01K17WBQ9G8PG95VMYZ4FYAEDZ',
 'movies:01K17WBQ9GEM6EE5SEVFYY7EDV',
 'movies:01K17WBQ9G5F1BZ47Q213FCDPY',
 'movies:01K17WBQ9GCEM3DF5XB1YYAG8W',
 'movies:01K17WBQ9G7C8RWA4AFHZ0WYJW',
 'movies:01K17WBQ9GZ399EY0ETKR2BWDC',
 'movies:01K17WBQ9GJSN56GAAEDTAQXCB',
 'movies:01K17WBQ9G1JPXZHW0VKFE1BG7',
 'movies:01K17WBQ9GWPPRG7W90W3JGX8K',
 'movies:01K17WBQ9GMCWGYG86P3CXCYGA',
 'movies:01K17WBQ9GB071J8B5AQD3PFK5',
 'movies:01K17WBQ9G3GZEBYMWM8X18YVA',
 'movies:01K17WBQ9GC7SXQ24KPDX0Y5P2']

## Search techniques

### Standard vector search

In [14]:
from redisvl.query import VectorQuery

user_query = "High tech and action packed movie"

embedded_user_query = hf.embed(user_query)

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "genre"],
    return_score=True,
)

result = index.query(vec_query)
pd.DataFrame(result)


,id,vector_distance,title,genre
0,movies:01K17WBQ9GNQXSNZ4Y152XMS2R,0.64973795414,Fast & Furious 9,action
1,movies:01K17WBQ9GFVP42C4RVN880ZTR,0.763235092163,Mad Max: Fury Road,action
2,movies:01K17WBQ9G7C8RWA4AFHZ0WYJW,0.792449712753,The Lego Movie,comedy


### Vector search with filters

Redis allows you to combine filter searches on fields within the index object allowing us to create more specific searches.

Search for top 3 movies specifically in the action genre:


In [15]:
from redisvl.query.filter import Tag

tag_filter = Tag("genre") == "action"

vec_query.set_filter(tag_filter)

result=index.query(vec_query)
pd.DataFrame(result)

,id,vector_distance,title,genre
0,movies:01K17WBQ9GNQXSNZ4Y152XMS2R,0.64973795414,Fast & Furious 9,action
1,movies:01K17WBQ9GFVP42C4RVN880ZTR,0.763235092163,Mad Max: Fury Road,action
2,movies:01K17WBQ9GS6XF9GDPFPSS342V,0.796153068542,Explosive Pursuit,action


Search for top 3 movies specifically in the action genre with ratings at or above a 7:


In [16]:
from redisvl.query.filter import Num

# build combined filter expressions
tag_filter = Tag("genre") == "action"
num_filter = Num("rating") >= 7
combined_filter = tag_filter & num_filter

# build vector query
vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "rating", "genre"],
    return_score=True,
    filter_expression=combined_filter
)

result = index.query(vec_query)
pd.DataFrame(result)

,id,vector_distance,title,rating,genre
0,movies:01K17WBQ9GFVP42C4RVN880ZTR,0.763235092163,Mad Max: Fury Road,8,action
1,movies:01K17WBQ9GS6XF9GDPFPSS342V,0.796153068542,Explosive Pursuit,7,action
2,movies:01K17WBQ9GEM6EE5SEVFYY7EDV,0.876494288445,Inception,9,action


Search with full text search for movies that directly mention "criminal mastermind" in the description:


In [17]:
from redisvl.query.filter import Text

text_filter = Text("description") % "criminal mastermind"

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "rating", "genre", "description"],
    return_score=True,
    filter_expression=text_filter
)

result = index.query(vec_query)
pd.DataFrame(result)

,id,vector_distance,title,rating,genre,description
0,movies:01K17WBQ9GB071J8B5AQD3PFK5,0.827254056931,Despicable Me,7,comedy,When a criminal mastermind uses a trio of orph...
1,movies:01K17WBQ9GJZCTSJ63Z4D75VX0,0.990856587887,The Dark Knight,9,action,"Batman faces off against the Joker, a criminal..."


Vector search with wildcard text match:


In [18]:
text_filter = Text("description") % "crim*"

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "rating", "genre", "description"],
    return_score=True,
    filter_expression=text_filter
)

result = index.query(vec_query)
pd.DataFrame(result)

,id,vector_distance,title,rating,genre,description
0,movies:01K17WBQ9GS6XF9GDPFPSS342V,0.796153068542,Explosive Pursuit,7,action,A daring cop chases a notorious criminal acros...
1,movies:01K17WBQ9GWPPRG7W90W3JGX8K,0.807471334934,The Incredibles,8,comedy,"A family of undercover superheroes, while tryi..."
2,movies:01K17WBQ9GB071J8B5AQD3PFK5,0.827254056931,Despicable Me,7,comedy,When a criminal mastermind uses a trio of orph...


Vector search with fuzzy match filter

> Note: fuzzy match is based on Levenshtein distance. Therefore, "hero" might return result for "her" as an example.

See docs for more info https://redis.io/docs/latest/develop/interact/search-and-query/advanced-concepts/query_syntax/


In [19]:

text_filter = Text("description") % "%hero%"

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "rating", "genre", "description"],
    return_score=True,
    filter_expression=text_filter
)

result = index.query(vec_query)
pd.DataFrame(result)

,id,vector_distance,title,rating,genre,description
0,movies:01K17WBQ9GD4RSBXDPFGJ5YMP6,0.889985740185,Black Widow,7,action,Natasha Romanoff confronts her dark past and f...
1,movies:01K17WBQ9G5F1BZ47Q213FCDPY,0.893866717815,The Avengers,8,action,Earth's mightiest heroes come together to stop...
2,movies:01K17WBQ9GC7SXQ24KPDX0Y5P2,0.943198204041,The Princess Diaries,6,comedy,Mia Thermopolis has just found out that she is...


### Range queries

Range queries allow you to set a pre defined distance "threshold" for which we want to return documents. This is helpful when you only want documents with a certain "radius" from the search query.

In [20]:
from redisvl.query import RangeQuery

user_query = "Family friendly fantasy movies"

embedded_user_query = hf.embed(user_query)

range_query = RangeQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    return_fields=["title", "rating", "genre"],
    return_score=True,
    distance_threshold=0.8  # find all items with a semantic distance of less than 0.8
)

result = index.query(range_query)
pd.DataFrame(result)


,id,vector_distance,title,rating,genre
0,movies:01K17WBQ9GWPPRG7W90W3JGX8K,0.64470243454,The Incredibles,8,comedy
1,movies:01K17WBQ9GD4RSBXDPFGJ5YMP6,0.747987031937,Black Widow,7,action
2,movies:01K17WBQ9GB071J8B5AQD3PFK5,0.750915527344,Despicable Me,7,comedy
3,movies:01K17WBQ9G1JPXZHW0VKFE1BG7,0.751298904419,Shrek,8,comedy
4,movies:01K17WBQ9GMCWGYG86P3CXCYGA,0.761669397354,"Monsters, Inc.",8,comedy
5,movies:01K17WBQ9GZ399EY0ETKR2BWDC,0.778580188751,Aladdin,8,comedy


Like the queries above, we can also chain additional filters and conditional operators with range queries. The following adds an `and` condition that returns vector search within the defined range and with a rating at or above 8.

In [21]:
range_query = RangeQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    return_fields=["title", "rating", "genre"],
    distance_threshold=0.8
)

numeric_filter = Num("rating") >= 8

range_query.set_filter(numeric_filter)

# in this case we want to do a simple filter search or the vector so we execute as a joint filter directly
result = index.query(range_query)
pd.DataFrame(result)


,id,vector_distance,title,rating,genre
0,movies:01K17WBQ9GWPPRG7W90W3JGX8K,0.64470243454,The Incredibles,8,comedy
1,movies:01K17WBQ9G1JPXZHW0VKFE1BG7,0.751298904419,Shrek,8,comedy
2,movies:01K17WBQ9GMCWGYG86P3CXCYGA,0.761669397354,"Monsters, Inc.",8,comedy
3,movies:01K17WBQ9GZ399EY0ETKR2BWDC,0.778580188751,Aladdin,8,comedy


### Full text search

In [22]:
from redisvl.query import TextQuery

user_query = "High tech, action packed, superheros fight scenes"

text_query = TextQuery(
    text=user_query,
    text_field_name="description",
    text_scorer="BM25STD",
    num_results=20,
    return_fields=["title", "description"],
)

result = index.query(text_query)[:4]
pd.DataFrame(result)[["title", "score"]]

,title,score
0,Fast & Furious 9,5.157032
1,The Incredibles,4.022877
2,Explosive Pursuit,2.335427
3,Toy Story,1.630097


### Hybrid search

In [23]:
from redisvl.query import HybridQuery

hybrid_query = HybridQuery(
    text=user_query,
    text_field_name="description",
    text_scorer="BM25",
    vector=embedded_user_query,
    vector_field_name="vector",
    alpha=0.7,
    num_results=20,
    return_fields=["title", "description"],
)

result = index.query(hybrid_query)[:4]
pd.DataFrame(result)[["title", "vector_similarity", "text_score", "hybrid_score"]]

,title,vector_similarity,text_score,hybrid_score
0,The Incredibles,0.67764878273,0.683368580998,0.67936472221
1,Fast & Furious 9,0.537397712469,0.498220622181,0.525644585383
2,Toy Story,0.553009659052,0.213523123792,0.451163698474
3,Black Widow,0.626006484032,0,0.438204538822


### Next steps

For more query examples with redisvl: [see here](https://github.com/redis/redis-vl-python/blob/main/docs/user_guide/02_hybrid_queries.ipynb)

In [24]:
# clean up!
index.delete()